In [1]:
import sys
sys.path.append('../../')

In [2]:
import random
random.seed(4444)
import pickle
import numpy as np
np.random.seed(5555)
import tensorflow as tf
tf.set_random_seed(6666)
import pandas as pd
from utils.dataset import DataSet, LABELS
from models.own import OwnSingleModel
from keras.utils import to_categorical
from sklearn.metrics import f1_score, accuracy_score
from keras.utils import plot_model

from env import *

%matplotlib inline

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
WORD_SEQS_PATH = os.path.join(CACHES_PATH, 'word_seqs_sw', 'simple')

TRAIN_SEQS_PADDED_PKL = os.path.join(WORD_SEQS_PATH, 'train_seqs_padded.pkl')
VAL_SEQS_PADDED_PKL = os.path.join(WORD_SEQS_PATH, 'val_seqs_padded.pkl')
TEST_SEQS_PADDED_PKL = os.path.join(WORD_SEQS_PATH, 'test_seqs_padded.pkl')

VECTOR_DIM = 300
EMBEDDING_PKL = os.path.join(WORD_SEQS_PATH, 'wem_%d.pkl' % VECTOR_DIM)

MODEL_FILE = os.path.join(SAVED_MODELS_PATH, 'own', 'single.h5')
RESULT_CSV = os.path.join(RESULTS_PATH, 'own_single.csv')

In [4]:
raw_dataset = DataSet()
train, val, test = raw_dataset.train, raw_dataset.val, raw_dataset.test

with open(TRAIN_SEQS_PADDED_PKL, 'rb') as f:
    train_seqs_padded = pickle.load(f)
    
with open(VAL_SEQS_PADDED_PKL, 'rb') as f:
    val_seqs_padded = pickle.load(f)
    
with open(TEST_SEQS_PADDED_PKL, 'rb') as f:
    test_seqs_padded = pickle.load(f)
    
with open(EMBEDDING_PKL, 'rb') as f:
    embedding = pickle.load(f)

In [5]:
train_with_seq = pd.merge(train, train_seqs_padded, on='id')
val_with_seq = pd.merge(val, val_seqs_padded, on='id')
test_with_seq = pd.merge(test, test_seqs_padded, on='id')

seq = 'words_seq'
train_x = np.array(list(train_with_seq[seq]))
val_x = np.array(list(val_with_seq[seq]))
test_x = np.array(list(test_with_seq[seq]))

y_cols = LABELS
val_ys = []
train_outputs = []
val_outputs = []
for col in y_cols:
    train_y = train_with_seq[col] + 2
    val_y = val_with_seq[col] + 2
    val_ys.append(val_y)
    train_y_onehot = to_categorical(train_y)
    val_y_onehot = to_categorical(val_y)
    train_outputs.append(train_y_onehot)
    val_outputs.append(val_y_onehot)

In [6]:
model = OwnSingleModel(max_len = train_x.shape[1], embedding = embedding)
model._model.summary()
plot_model(model._model)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 1123)         0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 1123, 300)    15000000    input_1[0][0]                    
__________________________________________________________________________________________________
spatial_dropout1d_1 (SpatialDro (None, 1123, 300)    0           embedding_1[0][0]                
__________________________________________________________________________________________________
bidirectional_1 (Bidirectional) (None, 1123, 600)    1083600     spatial_dropout1d_1[0][0]        
__________________________________________________________________________________________________
bidirectio

In [7]:
model.fit(train_x, train_outputs, 
          validation_data = (val_x, val_outputs),
          model_file = MODEL_FILE
         )

Train on 105000 samples, validate on 15000 samples
Epoch 1/300


ResourceExhaustedError: OOM when allocating tensor with shape[64,1123,300] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[Node: bidirectional_5/transpose_3 = Transpose[T=DT_FLOAT, Tperm=DT_INT32, _class=["loc:@training/Adam/gradients/bidirectional_5/transpose_3_grad/transpose"], _device="/job:localhost/replica:0/task:0/device:GPU:0"](bidirectional_5/CudnnRNN_1, training/Adam/gradients/bidirectional_11/transpose_grad/InvertPermutation)]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

	 [[Node: loss/dense_32_loss/Mean_2/_2339 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_21244_loss/dense_32_loss/Mean_2", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]
Hint: 

In [ ]:
model._model.load_weights(MODEL_FILE)
val_probs = model.predict(val_x)
test_probs = model.predict(test_x)

val_preds = list(map(lambda x: np.argmax(x, axis = -1), val_probs))
# val_preds = np.argmax(val_probs, axis = 1)
# test_preds = list(map(lambda x: np.argmax(x, axis = -1), test_probs))

In [ ]:
f1s = 0
for i, (vy, vp) in enumerate(zip(val_ys, val_preds)):
    f1 = f1_score(vy, vp, average='macro')
    print("The %sth f1: %s" % (i, f1))
    f1s += f1
    
print("The average f1 of val is %s" % (f1s / len(y_cols)))